
# Publicando Pacotes no PyPI

Neste tutorial você aprenderá como criar um projeto usando uv, instalar dependências, construir testes com mock e publicar como um pacote no pypi.org.

Os repositório do código pode ser encontrado nas referências no final do artigo.

## 1. Configurando o ambiente

### Instalando o uv

Para instalar o `uv` no seu sistema siga as instruções da [documentação do uv](https://docs.astral.sh/uv/getting-started/installation/) para seu sistema.

Por exemplo, para usuários linux, podemos instalar o uv da seguinte forma:

```sh
curl -LsSf https://astral.sh/uv/install.sh | sh
```

Podemos verificar a instalação do uv, com o comando

```sh
uv --version
```

Saída:

```sh
# uv 0.8.5
```

### Instalando o Python

Com o uv instalado você pode [instalar](https://docs.astral.sh/uv/guides/install-python/#getting-started) qualquer versão do python que desejar, por exemplo, para instalar o python 3.13 rode o comando

```sh
uv python install 3.13
```

Você pode instalar diversas versões do python no seu sistema, para tornar padrão uma versão específica, no uv, utilize o comando abaixo

```sh
uv python install 3.13 --default
```

### Instalando o vscode e plugins

Para este tutorial utilizaremos o [vscode](https://code.visualstudio.com/) com o plugins:

- Python: `ms-python.python`
- Pylance: `ms-python.vscode-pylance`
- Python Environments: `ms-python.vscode-python-envs`

## 2. Criando um Projeto

Para criar um projeto, no nosso caso, um projeto de lib de python utilizamos o comando init do uv

```sh
uv init --lib duden-webscraper
```
Após criar o projeto abra com o vscode

```sh
code duden-webscraper
```

você verá a segunte estrutura de arquivos


```sh
.
├── src
│   └── duden_webscraper
│       ├── __init__.py
│       └── py.typed
├── README.md
└── pyproject.toml
```

uma breve decrição de cada arquivo:

1. `./src/duden_webscraper/__init__.py`: O arquivo `__init__.py` é uma maneira de indicar ao Python que o diretório no qual ele se encontra deve ser tratado como um pacote (ou módulo).
2. `./src/duden_webscraper/py.typed`: Utilizado por analisadores estáticos como o mypy para determinar se o código do módulo está de acordo com as anotações de tipo especificadas, ajudando a encontrar erros de tipo em tempo de desenvolvimento. É geralmente um arquivo vazio, mas sua presença no diretório do módulo indica a compatibilidade com anotações de tipo.
3. `README.md`: É um arquivo markdown geralmente utilizado para documentação do pacote.
4. `pyproject.toml`: arquivo de configuração utilizado por ferramentas como o uv para declarar configurações do projeto. No caso do uv por exemplo, conterá informações sobre depedências, metadados do projeto como nome e descrição, versão do pacote e do python entre outras coisas.

## 3. Instalando dependências

O nosso projeto consiste de um webscraper que coleta informações do website duden.de a respeito de uma determinada palavra buscada. A ação implementada pode ser vista no diagrama abaixo

```{mermaid}
sequenceDiagram
    actor User
    participant Website as duden.de

    User->>Website: GET /rechtschreibung/{word}
    Website-->>User: metadata (e.g., lemma, POS, Fig, variants)
```

Para implementar a funcionalidade precisaremos dos pacotes:

1. [beautifulsoup4](https://pypi.org/project/beautifulsoup4/): pacote para manipulação de html.
2. [requests](https://pypi.org/project/requests/): pacote para execução de chamadas http.

Podemos instalar as depedências usando o uv com o comando abaixo:

```sh
uv add beautifulsoup4 requests
```

Neste ponto, você verá que ambos os pacotes foram adicionados no arquivo `pyproject.toml` e o uv criou um arquivo de controle de versionamento chamado `uv.lock`.

## 4. Criando o código

### Código

O código consiste dos arquivos:

1. `endpoint.py`: Contém constantes para a url do website.


```py
class Endpoint:
    BASE = "https://www.duden.de"
    DICTIONARY_SEARCH = "/search_api_autocomplete/dictionary_search"
    ORTHOGRAPHY = "/rechtschreibung"
```


2. `duden_web_scraper.py`: Contém a lógica de extração de informações do html obtido da consulta.


```py
import re
import requests
from bs4 import BeautifulSoup, Tag, NavigableString
from .endpoint import Endpoint
from .word_info_error import WordInfoError
from .word_not_found_error import WordNotFoundError

class DudenWebScraper:
    MIN_ATTEMPTS = 1

    def __init__(self):
        self.number_attempts = self.MIN_ATTEMPTS

    def get_word_info(self, word: str) -> dict:
        word = self.__convert(word)

        response = self.__get(Endpoint.ORTHOGRAPHY + "/" + word)

        if response is None:
            raise WordNotFoundError(f"the word {word} was not found")

        main = response.find('article')
        tuples_items = main.find_all(class_='tuple')
        tuples_items_contents = tuples_items[0].find(class_="tuple__val").text

        frequency = 0
        word_usage = None

        second_tuple_item = tuples_items[1].find(class_="tuple__key").text
        is_frequency = second_tuple_item.find("Häufigkeit") == 0
        frequency_position = 1 if is_frequency else 2

        if frequency_position == 2:
            word_usage = second_tuple_item.find(class_="tuple__val").text

        frequency_tuples = tuples_items[frequency_position].find(class_="tuple__val").find(class_="shaft__full")

        if len(frequency_tuples) > 0:
            frequency = frequency_tuples.text

        tuples_contents_pieces = tuples_items_contents.split(", ")
        word_gender = None

        if len(tuples_contents_pieces) > 1:
            word_gender = tuples_contents_pieces[1] if tuples_contents_pieces[1] is not None else None

        lemma = main.find(class_="lemma__main").text.replace("", "")
        determiner = main.find(class_="lemma__determiner")
        lemma_determiner = determiner.text if len(determiner) > 0 else None

        spelling_items = main.select("#rechtschreibung .tuple")
        spelling = self.__parse_spelling(spelling_items)

        meaning_items = main.select("#bedeutungen ol li")
        meaning = self.__parse_meanings(meaning_items)

        return {
            "lemma": lemma,
            "lemma_determiner": lemma_determiner,
            "word_type": tuples_contents_pieces[0],
            "word_usage": word_usage,
            "word_gender": word_gender,
            "frequency": frequency,
            "spelling": spelling,
            "meaning": meaning,
        }

    def __parse_meaning_kernel_tuples(self, tuples) -> list[dict]:
        results = []

        for note in tuples:
            results.append({
                "title": note.find(class_="tuple__key").text,
                "items": [
                    note.find(class_="tuple__val").text,
                ],
            })

        return results

    def __parse_meaning_kernel_notes(self, notes) -> list[dict]:
        results = []

        for note in notes:
            items = []

            for item in note.select(".note__list li"):
                items.append(item.text)
                results.append({
                    "title": note.find(class_="note__title").text,
                    "items": items,
                })

        return results

    def __parse_meaning_kernel(self, item) -> dict:
        parsedFigured = None
        figure = None
        enumeration_text = None
        notes = []
        tuples = []

        if not isinstance(item, NavigableString) and item.select("dl.note"):
            notes = item.select("dl.note")
        if not isinstance(item, NavigableString) and item.select("dl.tuple"):
            tuples = item.select("dl.tuple")
        if not isinstance(item, NavigableString) and item.find("figure"):
            figure = item.find("figure")
        if not isinstance(item, NavigableString) and item.find(class_="enumeration__text"):
            enumeration_text = item.find(class_="enumeration__text")
            enumeration_text = enumeration_text.text if len(enumeration_text) > 0 else None

        notes_list = []

        if len(notes) > 1:
            notes_list = self.__parse_meaning_kernel_notes(notes)
        else:
            notes_list = self.__parse_meaning_kernel_tuples(tuples)

        if figure:
            parsedFigured = {
                "link": figure.find("a")["href"],
                "caption": figure.find(class_="depiction__caption").text,
            }

        return {
            "text": enumeration_text,
            "figure": parsedFigured,
            "notes": notes_list,
        }

    def __parse_meanings(self, meanings) -> list:
        results = []

        for item in meanings:
            items = []
            sublists = item.find(class_="enumeration__sub-item")

            if sublists is None:
                continue

            if len(sublists) < 1:
                items.append(self.__parse_meaning_kernel(item))
                continue
            else:
                for sublist in sublists:
                    items.append(self.__parse_meaning_kernel(sublist))

            results.append(items)

        return results

    def __parse_spelling(self, spelling) -> (Tag | NavigableString | None):
        result = []

        for item in spelling:
            title = item.find(class_="tuple__key").text
            value = item.find(class_="tuple__val")
            result.append({
                "title": title,
                "value": value.text if title != "Verwandte Form" else value.find("a").text,
            })

        return result

    def __get(self, endpoint: str):
        response = requests.get(Endpoint.BASE + "/" + endpoint)

        if response.status_code != 200:
            raise WordInfoError("failed to get word info")

        soup = BeautifulSoup(response.content, "html.parser")
        return soup.find('body')

    def __convert(self, word: str) -> str:
        patterns = {
            '/ä/': 'ae',
            '/ö/': 'oe',
            '/ü/': 'ue',
            '/ß/': 'sz',
        }

        for pattern, replacement in patterns.items():
            word = re.sub(pattern, replacement, word)

        return word
```


3. `word_info_error.py`: Utilizado para lançamento de exceção quando o status de retorno da chamada http é diferente de 200.

```py
class WordInfoError(Exception):
    pass
```

4. `word_not_found_error.py`: Utilizado para lançamento de exceção caso o retorno da chamada http seja de sucesso mas o website retorne um resultado vazio.

```py
class WordNotFoundError(Exception):
    pass
```

A estrutura final de diretório é:

```sh
.
├── src
│   └── duden_webscraper
│       ├── __init__.py
│       ├── duden_web_scraper.py
│       ├── endpoint.py
│       ├── py.typed
│       ├── word_info_error.py
│       └── word_not_found_error.py
├── README.md
├── pyproject.toml
└── uv.lock
```

### Considerações

1. Os arquivos `word_info_error.py` e `word_not_found_error.py` são utilizados para especialização de erros. Ao construir pacotes é importante que cada tipo de erro tenha uma modelagem específica para que possa ser tratado de acordo com seu tipo.
2. no arquivo `duden_web_scraper.py` utilizamos métodos iniciando em `__` para indicar métodos privados. Como pode ser visto, existe somente um método público (método planejado para uso externo) `get_word_info`.

## 5. Testes com pytest

### Considerações

Para saber se o código que fizemos está funcionado corretamente, poderiamos publicar o pacote no pypi.org e instalar em uma outra aplicação python e usar. No entanto, esta é uma péssima prática.

A abordagem mais adequada é através de construição de testes unitários, funcionais e de integração:

1. **Testes unitários**: testes de entradas e saídas de métodos públicos geralmente as interações com dependência ou componentes externos são "mockadas", ou seja, os componentes que interagem com a unidade de código sendo testada são simulados.
2. **Testes E2E**: testes e2e (*end to end*) são testes onde o código é testado como uma caixa preta, possivelmente envolvendo interações com componentes externos. No nosso caso, os testes E2E farão requisições reais para o website duden.de.

Para construir os testes vamos utilizar o pacote [pytest](https://pypi.org/project/pytest/) que pode ser instalado com o uv através do comando

```sh
uv add --dev pytest
```

Note que como esta é uma dependência de desenvolvimento uitilizamos a flag `--dev`.

Para facilitar a simulação de requests nos testes funcionais também utilizaremos o pacote [requests-mock](https://pypi.org/project/requests-mock).

```sh
uv add --dev requests-mock
```

E finalmente para vermos quando das linhas estão cobertas por testes, utilizaremos a lib de cobertura de tests [pytest-cov](https://pypi.org/project/pytest-cov/).

```sh
uv add --dev pytest-cov
```

### Testes unitários

A estrutura do projeto até o momento é

```sh
.
├── src
│   └── duden_webscraper
│       ├── __init__.py
│       ├── duden_web_scraper.py
│       ├── endpoint.py
│       ├── py.typed
│       ├── word_info_error.py
│       └── word_not_found_error.py
├── tests
│   ├── e2e
│   ├── html
│   │   ├── Kaf.html
│   │   ├── Maskotte.html
│   │   ├── Pferd.html
│   │   └── Shop.html
│   └── unit
│       ├── test_duden_webscraper.py
│       └── test_endpoint.py
├── README.md
├── pyproject.toml
└── uv.lock
```

Como temos duas classes em nosso pacote vamos criar um arquivo de teste para cada uma dessas classes e testar seus métodos e constantes públicas.

O primeiro teste unitário é responsável por validar se o método `get_word_info` retorna o lema da palavra corretamente. Observe que o conteúdo HTML não é consultado diretamente no website, ao invés disso, a request é "mockada" para retornar um conteúdo simulado. A vantagem do uso de mocks está no tempo reduzido de execução do teste e na possibilidade de executar a validação da lógica do código independente do componente externo estar acessível ou não no ambiente onde os testes são executados.

```py
# tests/unit/test_duden_webscraper.py

from duden_webscraper.duden_web_scraper import DudenWebScraper
import pytest

def read_html(word):
    file_path = f"./tests/html/{word}.html"
    f = open(file_path, "r", encoding="utf-8")
    return f.read()

@pytest.mark.parametrize("word, status_code", [
    ("Pferd", 200),
    ("Maskotte", 200),
    ("Kaf", 200),
    ("Shop", 200),
])
def test_lemma_must_be_equals_to_word(requests_mock, word, status_code):
    mocked_url = f"https://www.duden.de/rechtschreibung/{word}"
    html_content = read_html(word)

    requests_mock.get(mocked_url, text=html_content, status_code=status_code)

    scraper = DudenWebScraper()
    result = scraper.get_word_info(word)

    assert result["lemma"] == word
```

O segundo teste unitário é bem mais simples e apenas valida se as constantes de URL definidas na classe endpoint tem os valores esperados.

```py
# tests/unit/test_endpoint.py

from duden_webscraper.endpoint import Endpoint
import pytest

@pytest.mark.parametrize(
    "attr, expected_prefix",
    [("DICTIONARY_SEARCH", "/"), ("ORTHOGRAPHY", "/")]
)
def test_paths_start_with_slash(attr, expected_prefix):
    assert getattr(Endpoint, attr).startswith(expected_prefix)
```

### Testes E2E

Em nosso caso temos somente um método com testes E2E. Neste caso, para que o teste execute com sucesso, é necessário conectividade do ambiente de testes com o website. Além disso, é necessário que o website esteja funcional no momento do teste.

```py
# tests/e2e/test_duden_webscraper_e2e.py

from duden_webscraper.duden_web_scraper import DudenWebScraper
import pytest

@pytest.mark.parametrize("word", [
    ("Pferd"),
    ("Maskotte"),
    ("Kaf"),
    ("Shop"),
])
def test_lemma_must_be_equals_to_word(word):
    scraper = DudenWebScraper()
    result = scraper.get_word_info(word)

    assert result["lemma"] == word
```

### Rodando os testes

Para rodar os testes utilize o comando na raíz do projeto

```sh
uv run -m pytest -s --cov=duden_webscraper --cov-report=term-missing
```

Saída:

```sh
===================================================== test session starts ======================================================
platform linux -- Python 3.13.4, pytest-8.4.1, pluggy-1.6.0
rootdir: /home/user/duden-webscraper
configfile: pyproject.toml
plugins: requests-mock-1.12.1, cov-6.2.1
collected 10 items                                                                                                             

tests/e2e/test_duden_webscraper_e2e.py ....
tests/unit/test_duden_webscraper.py ....
tests/unit/test_endpoint.py ..

======================================================== tests coverage ========================================================
_______________________________________ coverage: platform linux, python 3.13.4-final-0 ________________________________________

Name                                           Stmts   Miss  Cover   Missing
----------------------------------------------------------------------------
src/duden_webscraper/__init__.py                   0      0   100%
src/duden_webscraper/duden_web_scraper.py        107     41    62%   20, 36, 71-81, 84-96, 99-128, 144-151, 172
src/duden_webscraper/endpoint.py                   4      0   100%
src/duden_webscraper/word_info_error.py            2      0   100%
src/duden_webscraper/word_not_found_error.py       2      0   100%
----------------------------------------------------------------------------
TOTAL                                            115     41    64%
===================================================== 10 passed in 13.89s ======================================================
```

## 6. Publicando no PyPI

Para publicar no registro de pacotes pypi.org você precisará

1. Subir o código no github.
2. Ter uma conta no pypi.org.

Os passos para publicar um pacote são:

1. Rode o comando `uv build`, será gerado uma pasta dist que deve ser adicionada em seu .gitignore.
2. No pypi.org pegue o token.
3. Rode o comando `uv publish --token <seu-token>`.


## 7. Referências

1. [astral.sh](https://docs.astral.sh/uv/)
2. [pypi.org](https://pypi.org/)
3. [duden-webscraper](https://github.com/marciodojr/duden-webscraper)
4. [pytest.org](https://docs.pytest.org/en/stable/)